In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib import colors
from matplotlib.colors import LogNorm

%matplotlib inline
from scipy.optimize import curve_fit, minimize
import strax
import straxbra
from matplotlib.ticker import AutoMinorLocator
from matplotlib.image import NonUniformImage
import pandas as pd
# import tensorflow
import keras
from keras.models import Sequential,Model,load_model

Using TensorFlow backend.


In [2]:
model_NN = keras.models.load_model('XeBRA_Position_Reconstruction_NN_Model.h5')

In [5]:
run = '00178'





def CutCondition(integral, width, i):
    if integral[i] > 100 and integral[i] < 4000 and width[i] > 30 and width[i] < 50:
        return(True)
    else:
        return(False)
 
def S2_found(integral, width):
    #maximum_S2 = max(waveform)
    if integral > 500 and width >200 and width < 2000:
        return True
    else:
        return False
        
def reconstructed_position(input_array):
    ## Normalize sum input to 1 in order to correspond to area fraction in top array
    HFs_input = input_array / np.sum(input_array)
    ## Use model to reconstruct position
    ## Important: Factor 70 for rescaling label
    predictions = model_NN.predict(np.array(HFs_input))*70
    return predictions
    
def Process(waveform, i): 
    SelectionCutFlag = CutCondition(integral, width, i)
    if SelectionCutFlag == True:
        ###############now the S1 is found waveform[i]#############
        
        j = 1
        counter_timestamp = timestamp[i+j]
        threshold_timestamp = timestamp[i] + 45000
        
        #############as long as there is no waveform[i+j] which does not contain an S2 nor exceeds the time cut
                    # look at the next waveform (update j and the running timestamp)
        while threshold_timestamp >= counter_timestamp and  S2_found(integral = integral[i+j], width = width[i+j]) == False:  
            j += 1
            counter_timestamp = timestamp[i+j]
        else:

            if S2_found(integral = integral[i+j], width = width[i+j]):    
                k=1 
                s2_counter_timestamp = timestamp[i+j+k]
        ###########now an S2 is found in waveform[i+j] ---- check if there is an additional S2 within the time cut
                while threshold_timestamp >= s2_counter_timestamp and S2_found(integral = integral[i+j+k], width = width[i+j+k]) == False:
                    k += 1
                    s2_counter_timestamp = timestamp[i+j+k]
                else:
        ############ if there is no additional one within the drifttime window, save waveform[i+j] in a list
                    if threshold_timestamp <= s2_counter_timestamp:                             
                        cs_s2_area_corrected.append(integral[i+j])
                        cs_s2_width_corrected.append(width[i+j])
        ############# these are x , y positions based on 'peak_positions' ----- i guess this is what belongs to Alex    
                        cs_x_position.append(x_position[i+j])
                        cs_y_position.append(y_position[i+j])
                            


        ############# this is essential for the NN function by Alex: Input has to be an numpy (n,7) array
        ############# below we go through all top side PMTs. PMT_S2_area_list_corrected has the shape: list(7,n)
                        for m in range(len(PMT_S2_area_list_corrected)):
                            if area_per_channel[i+j][1] > 0 and area_per_channel[i+j][2] > 0 \
                            and area_per_channel[i+j][3] > 0 and area_per_channel[i+j][4] > 0 \
                            and area_per_channel[i+j][5] > 0 and area_per_channel[i+j][6] > 0 \
                            and area_per_channel[i+j][7] > 0 \
                            :
                                PMT_S2_area_list_corrected[m].append(area_per_channel[i+j][m+1])
                                   
                                

                            

                    
                return
            else:
                return

ctx = straxbra.XebraContext()

peaks_data = ctx.get_array(run, 'peaks')
position_data = ctx.get_array(run, 'peak_positions')
waveform = peaks_data['data']
integral = peaks_data['area']
width = peaks_data['width'][:,5]
timestamp = peaks_data['time']
area_per_channel = peaks_data['area_per_channel']
x_position = position_data['x']
y_position = position_data['y']
    

########## these are used for PR based on 'peak_positions' ---------- same as NN by Alex??????????     
cs_x_position = []
cs_y_position = []
  

cs_s2_area_corrected = []
cs_s2_width_corrected = []

                                                           
Area_s2_PMT1_corrected = []
Area_s2_PMT2_corrected = []
Area_s2_PMT3_corrected = []
Area_s2_PMT4_corrected = []
Area_s2_PMT5_corrected = []
Area_s2_PMT6_corrected = []
Area_s2_PMT7_corrected = []  
                                                                                                                                                                     
PMT_S2_area_list_corrected = [Area_s2_PMT1_corrected, Area_s2_PMT2_corrected,Area_s2_PMT3_corrected,Area_s2_PMT4_corrected,Area_s2_PMT5_corrected,Area_s2_PMT6_corrected,Area_s2_PMT7_corrected]  

     
                                                              
num = len(waveform)
print_counter = 0.0
for i in range(0,num):    
    Process(waveform,i)
    if i/num >= print_counter:
        print(round(print_counter,2), end = ' ')
        print_counter += 0.1
print(1.0)
    


####################the NN has to be implemented here! ########################   


0.0 0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1.0


In [11]:
print(len(PMT_S2_area_list_corrected[0]))


37723
[[4491.365, 2918.3943, 3072.1118, 707.9984, 2621.1187, 13763.794, 15725.995, 2024.644, 831.38086, 29117.068, 7089.104, 1472.7057, 1100.3741, 2101.53, 4336.9087, 2365.7837, 737.85376, 7664.606, 3932.0679, 10745.55, 4529.439, 4667.1304, 4442.915, 4337.628, 7854.2095, 6897.5728, 631.7323, 3399.6553, 1924.754, 5172.941, 1393.1859, 38.99494, 2113.7632, 1791.6647, 4520.328, 26.016405, 7433.0103, 2596.3096, 2574.7056, 6790.6665, 8776.338, 1455.1091, 3842.4238, 2844.3164, 631.4615, 2430.8381, 1378.6299, 1729.611, 1412.2367, 238.30186, 2984.7322, 929.005, 867.5072, 5.9509735, 1296.8475, 5310.761, 2753.8289, 223.85667, 2922.0115, 7033.891, 946.945, 4586.1157, 3562.7693, 16997.344, 322.88535, 911.951, 5681.58, 7848.1147, 37438.1, 10472.962, 5632.61, 5107.653, 2755.0266, 3236.3972, 8941.568, 3895.555, 994.35333, 8794.516, 763.51154, 28422.281, 1042.4661, 8585.636, 408.65485, 3949.25, 8943.18, 385.414, 270.58047, 2338.5361, 2176.789, 1859.5132, 29752.549, 2864.0105, 36858.266, 2931.764, 796.5

In [13]:
input_array = np.array(PMT_S2_area_list_corrected)
input_array_final = input_array.T
positions = reconstructed_position(input_array_final)
x_position_raw = positions[:,0]
y_position_raw = positions[:,1]  
print(input_array.shape)
print(input_array_final.shape)

(7, 37723)
(37723, 7)


In [15]:
x_position_raw

array([-0.3572715 , -0.3568913 , -0.35821185, ..., -0.35748953,
       -0.3568793 , -0.35740033], dtype=float32)